In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt

# Module relatif à Gurobi
from gurobipy import *

In [2]:
model = Model("SNCF JALON 1")

N = 5           # Nombre de wagons
M = 7           # Nombre de tâches à effectuer sur chaque wagon
S = 60*24*7     # Nombre de minutes dans une semaine    

t = model.addMVar(shape = (M,N), name = "t",vtype = GRB.INTEGER) # Temps de début de la tâche m sur le wagon n, en minute, comptée à partir du lundi 8 Aout 2022 00:00
delta_1 = model.addMVar(shape = (M,N,N,2), name = "delta_1", vtype=GRB.BINARY)
delta_2 = model.addMVar(shape = (M,N,11), name = "delta_2", vtype=GRB.BINARY)


Restricted license - for non-production use only - expires 2026-11-23


In [3]:
T = np.array([15,45,15,15,150,15,20])       # Durée des tâches
t_a = None                                  # temps d'arrivée des wagons
t_d = None                                  # temps de départ des wagons

**Contraintes de temporalité des tâches sur un même wagon et respect des heures de départ et d'arrivée**

In [4]:
for n in range(N):
    model.addConstr(t[0,n] >= t_a[n])               # Début de la première tâche intervient après l'arrivée du wagon
    model.addConstr(t[M-1,n] + T[M-1] <= t_d[n])    # Fin de la dernière tâche avant le départ du wagon
    for m in range(M-1):
        model.addConstr(t[m,n] + T[m] <= t[m+1,n])  # Début tâche m+1 après la fin de la tâche m

TypeError: 'NoneType' object is not subscriptable

**Contrainte permettant d'avoir au plus un wagon par machine à chaque instant**

In [ ]:
for m in [2,4,5]:
    for n1 in range(N-1):
        for n2 in range(n1+1,N):
            model.addGenConstrIndicator(delta_1[m,n1,n2,0], True, t[m,n2] <= t[m,n1] - T[m])    # Fin tâche wagon n2 avant début tâche wagon n1
            model.addGenConstrIndicator(delta_1[m,n1,n2,1], True, t[m,n2] >= t[m,n1] + T[m])    # Début tâche wagon n2 après fin tâche wagon n1
            model.addConstr(delta_1[m,n1,n2,0] + delta_1[m,n1,n2,1] >= 1)                       # Un condition ou l'autre pour éciter le recouvrement

**Contrainte du respect de l'ouverture du chantier de formation et de la disponibilité des machines**

In [6]:
Limites = np.array([5,13,5*24+13,5*24+21,6*24+13,6*24+21,7*24+5,7*24+13])*60       # Horaires limites de la disponibilité du chantier de formation et des machines (en minutes)

for m in [2,3,4,5]:
    for n in range(N):
        model.addGenConstrIndicator(delta_2[m,n,0], True, t[m,n] <= Limites[0] - T[m])  # Fin de la tâche m avant lundi 08/08 5h
        model.addGenConstrIndicator(delta_2[m,n,1], True, t[m,n] >= Limites[1])         # Début de la tâche m après lundi 08/08 13h
        model.addGenConstrIndicator(delta_2[m,n,2], True, t[m,n] <= Limites[2] - T[m])  # Fin de la tâche m avant samedi 13/08 13h
        model.addGenConstrIndicator(delta_2[m,n,3], True, t[m,n] >= Limites[3])         # Début de la tâche m après samedi 13/08 21h
        model.addGenConstrIndicator(delta_2[m,n,4], True, t[m,n] <= Limites[4]- T[m])   # Fin de la tâche m avant dimanche 14/08 13h
        model.addGenConstrIndicator(delta_2[m,n,5], True, t[m,n] >= Limites[5])         # Début de la tâche m après dimanche 14/08 21h
        model.addGenConstrIndicator(delta_2[m,n,6], True, t[m,n] <= Limites[6] - T[m])  # Fin de la tâche m avant lundi 15/08 5h
        model.addGenConstrIndicator(delta_2[m,n,7], True, t[m,n] >= Limites[7])         # Début de la tâche m après lundi 15/08 13h
        
        model.addConstr(delta_2[m,n,8] == delta_2[m,n,1] * delta_2[m,n,2])              # Condition 2 et Condition 3
        model.addConstr(delta_2[m,n,9] == delta_2[m,n,3] * delta_2[m,n,4])              # Condition 4 et Condition 5
        model.addConstr(delta_2[m,n,10] == delta_2[m,n,5] * delta_2[m,n,6])             # Condition 6 et Condition 6
        
        model.addConstr(delta_2[m,n,0] + delta_2[m,n,8] + delta_2[m,n,9] + delta_2[m,n,10] + delta_2[m,n,7] >= 1) # C1 ou (C2 et C3) ou (C4 et C5) ou (C6 et C7) ou C8
        